### Introduction

#### Every document we read can be thought of as consisting of many topics all stacked upon one another. Today, we’re going can unpack these topics using of NLP techniques: 
- Latent Dirichlet Allocation (LDA) and Topic Modeling
- Data is collected on https://www.reuters.com/breakingviews by a scrapping script
- The goal is to break text documents down into topics by word. 
- What is laten feature ? Mathematically, we want to find “topics” that are collections of words that appear in similar documents. 
  More generally, it is a collection of features in a dataset.
- There are several libraries for LDA such as scikit-learn and gensim. I choose gensim for this project. 

#### Project tasks:
- Cleaning the dataset & Lemmatization
- Creat a dictionay from processed data
- Create Corpus and LDA Model with bag of words
- Create Coprpus and LDA with TF-IDF
- Caculate the Perplexity and Topic Cohenrence between two models
- Visualize topics with the help of pyLDAvis


####  Import libraries

In [85]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
import nltk
nltk.download('wordnet')
import string
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize
import pandas as pd
import unidecode

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline



[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\btdiem\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [86]:
import pandas as pd
data = pd.read_csv('../data/breakingnews.csv', error_bad_lines=False);

In [87]:
data.head(2)

,Unnamed: 0,headline
0,0,Joseph Stiglitz does what he does very well. T...
1,1,For all the successes of Japanese Prime Minist...


In [88]:
data_text = data[['headline']]
data_text['index'] = data_text.index
documents = data_text

#####  Preprocessing Data & Lemmatization

In [89]:
stemmer = SnowballStemmer('english')

def get_wordnet_pos(treebank_tag):
    """Convert the part-of-speech naming scheme
       from the nltk default to that which is
       recognized by the WordNet lemmatizer"""

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

      
# remove alpha numerical words and make lowercase
alphanum_re = re.compile(r"""\w*\d\w*""")
alphanum_lambda = lambda x: alphanum_re.sub('', x)

re_alpha = re.compile('[^A-Za-z]', re.UNICODE)
alphaonly = lambda x : re_alpha.sub(' ', x)

# remove punctuation
punc_re = re.compile('[%s]' % re.escape(string.punctuation))
punc_lambda = lambda x: punc_re.sub(' ', x)

single_quote1 = re.compile("’")
nosinglequote1 = lambda x : re.sub(single_quote1 , '', x)

single_quote2 = re.compile('‘*')
nosinglequote2 = lambda x : re.sub(single_quote2 , '', x)


double_quote = re.compile('["]*')
nodoublequote = lambda x : re.sub(double_quote , '', x)

# remove stop words
sw = stopwords.words('english')
sw_lambda = lambda x: list(filter(lambda y: y not in sw, x))

pos_lambda = lambda x: [(y[0], get_wordnet_pos(y[1])) for y in x]

lemmatizer = WordNetLemmatizer()
lem_lambda = lambda x: [lemmatizer.lemmatize(*y) for y in x]



def preprocess_raw_data(data):
    """
    data: Pandas series
    """
     # remove email
    email_re =  re.compile('\S*@\S*\s?')
    noemail = lambda x : email_re.sub(' ', x)
    data = data.map(noemail)
 
    # remove new line character:
    newline_re = re.compile('\s+')
    nonewline = lambda x : newline_re.sub(' ', x)
    data = data.map(nonewline)
    # Remove distracting single quotes
    sg_quote_re = re.compile("\'")
    no_sg_quote = lambda x : sg_quote_re.sub(' ', x)
    data = data.map(no_sg_quote)
    
    data = data.map(simple_preprocess)
    
    # remove stop words
#    data = data.map(word_tokenize)
    sw = stopwords.words('english')
    sw_lambda = lambda x: list(filter(lambda y: y not in sw, x))
    # tokenize words before removing stopwords
    data = data.map(sw_lambda)

    # part of speech tagging--must convert to format used by lemmatizer
    data = data.map(nltk.pos_tag)
    data = data.map(pos_lambda)
    # lemmatization
    data = data.map(lem_lambda)
    
    return data
 
def get_score(lda_model, doc2vec):
    """
    lda_model: LDA model 
    
    """
    for index, score in sorted(lda_model[doc2vec], key=lambda tup: -1*tup[1]):
        print("\nScore: {}\nTopic: {} \nWord: {}".format(score, index, lda_model.print_topic(index, 10)))


<input>:57: DeprecationWarning: invalid escape sequence \S
<input>:62: DeprecationWarning: invalid escape sequence \s
<input>:57: DeprecationWarning: invalid escape sequence \S
<input>:62: DeprecationWarning: invalid escape sequence \s
<input>:57: DeprecationWarning: invalid escape sequence \S
<input>:62: DeprecationWarning: invalid escape sequence \s
<ipython-input-89-422b4386cee7>:57: DeprecationWarning: invalid escape sequence \S
  email_re =  re.compile('\S*@\S*\s?')
<ipython-input-89-422b4386cee7>:62: DeprecationWarning: invalid escape sequence \s
  newline_re = re.compile('\s+')


In [90]:
processed_docs = preprocess_raw_data(documents['headline'])

#### Create the Dictionary and Corpus

In [91]:
# Create a corpus from a list of texts
dictionary = corpora.Dictionary(processed_docs)
# filter out the less common words
# Keep tokens which are contained in at least 15 documents
# Keep tokens which are contained in no more than 50% documents
# Keep only the first 10000 most frequent tokens
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=10000)
# Term Document Frequency, it is a list of (word_id, word_frequency) in the processed_docs.
corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
# View the first document in corpus
print(corpus[:1])

[[(0, 1), (1, 1)]]


In [92]:
#see what words from given ids in dictionary and their frequency
[[(dictionary[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('may', 1), ('provide', 1)]]

#### LDA Topic Modeling (Bag of words)
- Building LDA using Bag of Words with 5 topics
- LDA model is built with 5 different topics where each topic is a combination of keywords and each keyword contributes a certain weightage to the topic.

In [93]:
lda_model = gensim.models.LdaMulticore(corpus
                                       , num_topics=5
                                       , id2word=dictionary
                                       , iterations=50
                                       , passes=2
                                       , workers=4)


##### View the topic

In [94]:
#pprint(lda_model.print_topics())
for idx, topic in lda_model.print_topics():
    print('\nTopic: {}\nWords: {}'.format(idx+1, topic))



Topic: 0
Words: 0.647*"year" + 0.086*"may" + 0.052*"billion" + 0.051*"news" + 0.051*"provide" + 0.051*"big" + 0.021*"world" + 0.021*"com" + 0.020*"day"

Topic: 1
Words: 0.245*"world" + 0.196*"news" + 0.147*"day" + 0.147*"com" + 0.099*"provide" + 0.057*"billion" + 0.057*"big" + 0.050*"may" + 0.002*"year"

Topic: 2
Words: 0.404*"may" + 0.401*"provide" + 0.043*"year" + 0.042*"big" + 0.042*"news" + 0.018*"com" + 0.017*"world" + 0.017*"billion" + 0.017*"day"

Topic: 3
Words: 0.317*"billion" + 0.315*"may" + 0.170*"com" + 0.169*"year" + 0.006*"big" + 0.006*"news" + 0.006*"provide" + 0.006*"world" + 0.006*"day"

Topic: 4
Words: 0.349*"big" + 0.183*"news" + 0.182*"year" + 0.182*"billion" + 0.048*"may" + 0.036*"provide" + 0.007*"com" + 0.007*"world" + 0.007*"day"


#### Compute Model Perplexity and Coherence Score

In [95]:
# Compute Perplexity
# a measure of how good the model is. the lower, the better.
print('\nPerplexity: ', lda_model.log_perplexity(corpus)) 

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model
                                     , corpus=corpus
                                     , texts = list(processed_docs)
                                     , dictionary=dictionary 
                                     ,coherence='c_v')

coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -2.5590968742763436

Coherence Score:  0.4224021185538137


#### Visualize the topics-keywords

In [96]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
vis

C:\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.208348 -0.069882       1        1  55.292545
3     -0.112796  0.091832       2        1  14.730565
4     -0.052608 -0.137204       3        1  14.699700
2      0.084261  0.153031       4        1   9.598559
0     -0.127206 -0.037777       5        1   5.678626, topic_info=  Category       Freq     Term      Total  loglift  logprob
7  Default  19.000000     year  19.000000   9.0000   9.0000
1  Default  20.000000  provide  20.000000   8.0000   8.0000
0  Default  25.000000      may  25.000000   7.0000   7.0000
8  Default  18.000000      big  18.000000   6.0000   6.0000
5  Default  22.000000  billion  22.000000   5.0000   5.0000
3  Default  29.000000     news  29.000000   4.0000   4.0000
6  Default  22.000000      com  22.000000   3.0000   3.0000
4  Default  28.000000    world  28.000000   2.0000   2.0000
2  Default  17.000000      day  17.000000   1.0000   1.0000
4   Topic1  27.589518    world  28.537748   0.5587  -1.4082
2   Topic1  16.623343      day  17.558657   0.5378  -1.9148
3   Topic1  22.104799     news  29.163826   0.3154  -1.6298
6   Topic1  16.618422      com  22.497549   0.2896  -1.9151
1   Topic1  11.128611  provide  20.824787  -0.0341  -2.3161
8   Topic1   6.467888      big  18.524179  -0.4597  -2.8587
5   Topic1   6.484293  billion  22.422827  -0.6482  -2.8562
0   Topic1   5.583499      may  25.395643  -0.9222  -3.0058
7   Topic1   0.196421     year  19.074772  -3.9833  -6.3531
5   Topic2   9.537374  billion  22.422827   1.0604  -1.1477
0   Topic2   9.480559      may  25.395643   0.9299  -1.1536
7   Topic2   5.083280     year  19.074772   0.5928  -1.7769
6   Topic2   5.094484      com  22.497549   0.4300  -1.7747
2   Topic2   0.168017      day  17.558657  -2.7340  -5.1866
8   Topic2   0.173437      big  18.524179  -2.7558  -5.1548
1   Topic2   0.171517  provide  20.824787  -2.8840  -5.1659
4   Topic2   0.169942    world  28.537748  -3.2083  -5.1752
3   Topic2   0.171742     news  29.163826  -3.2194  -5.1646
8   Topic3  10.470394      big  18.524179   1.3468  -1.0522
7   Topic3   5.460446     year  19.074772   0.6665  -1.7032
5   Topic3   5.455942  billion  22.422827   0.5040  -1.7041
3   Topic3   5.475939     news  29.163826   0.2448  -1.7004
0   Topic3   1.431248      may  25.395643  -0.9587  -3.0422
1   Topic3   1.087077  provide  20.824787  -1.0353  -3.3173
2   Topic3   0.200732      day  17.558657  -2.5540  -5.0066
6   Topic3   0.203228      com  22.497549  -2.7895  -4.9942
4   Topic3   0.202383    world  28.537748  -3.0315  -4.9984
1   Topic4   7.846518  provide  20.824787   1.3675  -0.9145
0   Topic4   7.909016      may  25.395643   1.1770  -0.9066
8   Topic4   0.821456      big  18.524179  -0.7722  -3.1712
7   Topic4   0.839962     year  19.074772  -0.7792  -3.1490
3   Topic4   0.815207     news  29.163826  -1.2337  -3.1789
2   Topic4   0.329165      day  17.558657  -1.6332  -4.0858
6   Topic4   0.343754      com  22.497549  -1.8377  -4.0424
5   Topic4   0.337927  billion  22.422827  -1.8514  -4.0595
4   Topic4   0.338057    world  28.537748  -2.0922  -4.0591
7   Topic5   7.494663     year  19.074772   1.9343  -0.4355
0   Topic5   0.991321      may  25.395643  -0.3748  -2.4584
8   Topic5   0.591004      big  18.524179  -0.5765  -2.9756
1   Topic5   0.591065  provide  20.824787  -0.6935  -2.9755
5   Topic5   0.607290  billion  22.422827  -0.7404  -2.9484
3   Topic5   0.596141     news  29.163826  -1.0217  -2.9669
2   Topic5   0.237402      day  17.558657  -1.4351  -3.8877
6   Topic5   0.237661      com  22.497549  -1.6819  -3.8866
4   Topic5   0.237849    world  28.537748  -1.9189  -3.8858, token_table=      Topic      Freq     Term
term                          
8         1  0.323901      big
8         3  0.539835      big
8         4  0.053983      big
8         5  0.053983      big
5         1  0.267584  billion
5         2  0.445974  billion
5         3  0.222

#### LDA model with TF-IDF


In [97]:
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

In [98]:
[[(dictionary[id], freq) for id, freq in cp] for cp in corpus_tfidf[:1]]

[[('may', 0.6309418522375851), ('provide', 0.7758301225751714)]]

In [99]:
lda_tfidf = gensim.models.LdaMulticore(corpus_tfidf
                                       , num_topics=5
                                       , id2word=dictionary
                                       , iterations=50)

#### View the topic

In [100]:
#pprint(lda_model.print_topics())
for idx, topic in lda_tfidf.print_topics():
    print('\nTopic: {}\nWords: {}'.format(idx+1, topic))



Topic: 1
Words: 0.484*"may" + 0.200*"billion" + 0.170*"provide" + 0.067*"big" + 0.017*"year" + 0.015*"world" + 0.015*"com" + 0.015*"news" + 0.015*"day"

Topic: 2
Words: 0.521*"year" + 0.200*"news" + 0.200*"big" + 0.015*"may" + 0.014*"provide" + 0.013*"billion" + 0.012*"world" + 0.012*"com" + 0.012*"day"

Topic: 3
Words: 0.317*"may" + 0.247*"billion" + 0.177*"provide" + 0.177*"big" + 0.018*"year" + 0.016*"world" + 0.016*"com" + 0.016*"news" + 0.016*"day"

Topic: 4
Words: 0.193*"billion" + 0.190*"com" + 0.146*"year" + 0.142*"may" + 0.093*"world" + 0.089*"big" + 0.069*"day" + 0.047*"news" + 0.031*"provide"

Topic: 5
Words: 0.243*"world" + 0.214*"news" + 0.158*"day" + 0.130*"com" + 0.092*"provide" + 0.062*"may" + 0.046*"big" + 0.046*"billion" + 0.008*"year"


#### Compute Perplexity & Coherence Score

In [101]:
# Compute Perplexity
# a measure of how good the model is. the lower, the better.
print('\nPerplexity: ', lda_tfidf.log_perplexity(corpus_tfidf)) 

# Compute Coherence Score
coherence_model_tfidf = CoherenceModel(model=lda_tfidf
                                     , corpus=corpus_tfidf
                                     , texts = list(processed_docs)
                                     , dictionary=dictionary 
                                     ,coherence='c_v')

coherence_tfidf = coherence_model_tfidf.get_coherence()
print('\nCoherence Score: ', coherence_tfidf)



Perplexity:  -3.186376803651969

Coherence Score:  0.4224021185538137


'\nTopics = 15\n\nPerplexity:  -9.27488119057329\n\nCoherence Score:  0.24405747657214635\n\nTopics = 10\n\nPerplexity:  -9.00745369196948\n\nCoherence Score:  0.2058747841509882\n\nTopics = 20\n\nPerplexity:  -9.656468835436096\n\nCoherence Score:  0.3316541111751868\n\n'

#### Visualize the topics

In [102]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_tfidf, corpus_tfidf, dictionary)
vis

C:\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.026096  0.174960       1        1  33.870201
0      0.178974 -0.042599       2        1  21.097637
1     -0.260958 -0.079541       3        1  17.674608
3     -0.017757  0.013375       4        1  14.493519
2      0.125836 -0.066194       5        1  12.864039, topic_info=  Category       Freq     Term      Total  loglift  logprob
7  Default  11.000000     year  11.000000   9.0000   9.0000
0  Default  17.000000      may  17.000000   8.0000   8.0000
5  Default  11.000000  billion  11.000000   7.0000   7.0000
6  Default   7.000000      com   7.000000   6.0000   6.0000
4  Default   9.000000    world   9.000000   5.0000   5.0000
3  Default  11.000000     news  11.000000   4.0000   4.0000
1  Default   9.000000  provide   9.000000   3.0000   3.0000
8  Default   9.000000      big   9.000000   2.0000   2.0000
2  Default   6.000000      day   6.000000   1.0000   1.0000
4   Topic1   7.877998    world   9.882693   0.8559  -1.4143
2   Topic1   5.117869      day   6.783290   0.8009  -1.8456
3   Topic1   6.936696     news  11.482994   0.5786  -1.5416
6   Topic1   4.227954      com   7.579454   0.4989  -2.0367
1   Topic1   2.981504  provide   9.249574  -0.0495  -2.3860
8   Topic1   1.497076      big   9.648666  -0.7807  -3.0749
5   Topic1   1.483777  billion  11.462650  -0.9619  -3.0838
0   Topic1   2.013323      may  17.907013  -1.1028  -2.7786
7   Topic1   0.271159     year  11.684697  -2.6807  -4.7834
0   Topic2   9.771458      may  17.907013   0.9503  -0.7255
1   Topic2   3.429775  provide   9.249574   0.5639  -1.7725
5   Topic2   4.037701  billion  11.462650   0.5126  -1.6093
8   Topic2   1.360980      big   9.648666  -0.4026  -2.6968
2   Topic2   0.304703      day   6.783290  -1.5469  -4.1934
6   Topic2   0.311125      com   7.579454  -1.6370  -4.1726
4   Topic2   0.312750    world   9.882693  -1.8971  -4.1674
7   Topic2   0.347065     year  11.684697  -1.9605  -4.0633
3   Topic2   0.310880     news  11.482994  -2.0532  -4.1734
7   Topic3   8.815497     year  11.684697   1.4513  -0.6515
8   Topic3   3.384223      big   9.648666   0.6853  -1.6089
3   Topic3   3.384805     news  11.482994   0.5115  -1.6087
2   Topic3   0.205721      day   6.783290  -1.7627  -4.4092
6   Topic3   0.208292      com   7.579454  -1.8612  -4.3968
1   Topic3   0.230071  provide   9.249574  -1.9609  -4.2973
4   Topic3   0.210315    world   9.882693  -2.1169  -4.3871
5   Topic3   0.222339  billion  11.462650  -2.2096  -4.3315
0   Topic3   0.249983      may  17.907013  -2.5385  -4.2143
6   Topic4   2.634326      com   7.579454   0.8747  -1.6609
5   Topic4   2.673433  billion  11.462650   0.4757  -1.6462
7   Topic4   2.027501     year  11.684697   0.1800  -1.9227
2   Topic4   0.962599      day   6.783290  -0.0211  -2.6677
4   Topic4   1.283512    world   9.882693  -0.1097  -2.3800
8   Topic4   1.230937      big   9.648666  -0.1276  -2.4218
0   Topic4   1.970823      may  17.907013  -0.2753  -1.9511
3   Topic4   0.657996     news  11.482994  -0.9280  -3.0481
1   Topic4   0.426419  provide   9.249574  -1.1454  -3.4819
5   Topic5   3.045400  billion  11.462650   0.7253  -1.3967
1   Topic5   2.181805  provide   9.249574   0.6063  -1.7301
8   Topic5   2.175451      big   9.648666   0.5612  -1.7330
0   Topic5   3.901426      may  17.907013   0.5269  -1.1489
2   Topic5   0.192397      day   6.783290  -1.5119  -4.1585
6   Topic5   0.197756      com   7.579454  -1.5954  -4.1310
4   Topic5   0.198118    world   9.882693  -1.8589  -4.1292
7   Topic5   0.223474     year  11.684697  -1.9060  -4.0087
3   Topic5   0.192617     news  11.482994  -2.0372  -4.1573, token_table=      Topic      Freq     Term
term                          
8         1  0.103641      big
8         2  0.103641      big
8         3  0.310924      big
8         4  0.103641      big
8         5  0.207283      big
5         1  0.087240  billion
5         2  0.348

In [106]:
for idx, topic in lda_tfidf.print_topics(-1):
    print('\nTopic: {}\nWords: {}'.format(idx+1, topic))


Topic: 1
Words: 0.484*"may" + 0.200*"billion" + 0.170*"provide" + 0.067*"big" + 0.017*"year" + 0.015*"world" + 0.015*"com" + 0.015*"news" + 0.015*"day"

Topic: 2
Words: 0.521*"year" + 0.200*"news" + 0.200*"big" + 0.015*"may" + 0.014*"provide" + 0.013*"billion" + 0.012*"world" + 0.012*"com" + 0.012*"day"

Topic: 3
Words: 0.317*"may" + 0.247*"billion" + 0.177*"provide" + 0.177*"big" + 0.018*"year" + 0.016*"world" + 0.016*"com" + 0.016*"news" + 0.016*"day"

Topic: 4
Words: 0.193*"billion" + 0.190*"com" + 0.146*"year" + 0.142*"may" + 0.093*"world" + 0.089*"big" + 0.069*"day" + 0.047*"news" + 0.031*"provide"

Topic: 5
Words: 0.243*"world" + 0.214*"news" + 0.158*"day" + 0.130*"com" + 0.092*"provide" + 0.062*"may" + 0.046*"big" + 0.046*"billion" + 0.008*"year"


In [104]:
#Make a test
print(corpus[10])
[print(dictionary[id], freq) for id, freq in corpus[10]]

get_score(lda_model, corpus[10])
#for index, score in sorted(lda_model[common_corpus[4310]], key=lambda tup: -1*tup[1]):
#    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))

[(1, 1), (2, 1), (4, 2), (6, 2), (8, 1)]
provide 1
day 1
world 2
com 2
big 1

Score: 0.89858078956604
Topic: 1 
Word: 0.245*"world" + 0.196*"news" + 0.147*"day" + 0.147*"com" + 0.099*"provide" + 0.057*"billion" + 0.057*"big" + 0.050*"may" + 0.002*"year"

Score: 0.025722915306687355
Topic: 4 
Word: 0.349*"big" + 0.183*"news" + 0.182*"year" + 0.182*"billion" + 0.048*"may" + 0.036*"provide" + 0.007*"com" + 0.007*"world" + 0.007*"day"

Score: 0.025433992967009544
Topic: 2 
Word: 0.404*"may" + 0.401*"provide" + 0.043*"year" + 0.042*"big" + 0.042*"news" + 0.018*"com" + 0.017*"world" + 0.017*"billion" + 0.017*"day"

Score: 0.025217458605766296
Topic: 3 
Word: 0.317*"billion" + 0.315*"may" + 0.170*"com" + 0.169*"year" + 0.006*"big" + 0.006*"news" + 0.006*"provide" + 0.006*"world" + 0.006*"day"

Score: 0.025044839829206467
Topic: 0 
Word: 0.647*"year" + 0.086*"may" + 0.052*"billion" + 0.051*"news" + 0.051*"provide" + 0.051*"big" + 0.021*"world" + 0.021*"com" + 0.020*"day"


In [105]:

print(corpus_tfidf[10])
[print(dictionary[id], freq) for id, freq in corpus_tfidf[10]]

get_score(lda_tfidf, corpus_tfidf[10])
    

[(1, 0.30151134457776363), (2, 0.30151134457776363), (4, 0.6030226891555273), (6, 0.6030226891555273), (8, 0.30151134457776363)]
provide 0.30151134457776363
day 0.30151134457776363
world 0.6030226891555273
com 0.6030226891555273
big 0.30151134457776363

Score: 0.7362167835235596
Topic: 4 
Word: 0.243*"world" + 0.214*"news" + 0.158*"day" + 0.130*"com" + 0.092*"provide" + 0.062*"may" + 0.046*"big" + 0.046*"billion" + 0.008*"year"

Score: 0.06635069847106934
Topic: 3 
Word: 0.193*"billion" + 0.190*"com" + 0.146*"year" + 0.142*"may" + 0.093*"world" + 0.089*"big" + 0.069*"day" + 0.047*"news" + 0.031*"provide"

Score: 0.06631165742874146
Topic: 2 
Word: 0.317*"may" + 0.247*"billion" + 0.177*"provide" + 0.177*"big" + 0.018*"year" + 0.016*"world" + 0.016*"com" + 0.016*"news" + 0.016*"day"

Score: 0.06595547497272491
Topic: 1 
Word: 0.521*"year" + 0.200*"news" + 0.200*"big" + 0.015*"may" + 0.014*"provide" + 0.013*"billion" + 0.012*"world" + 0.012*"com" + 0.012*"day"

Score: 0.06516535580158234
